In [19]:
import yfinance as yf
import numpy as np
import pandas as pd
from get_data_yf import get_symbols

#TODO writes advised company recommendation data to an excel file

RECOMMENDATION_WEIGHTS = [2, 1, 0, -1, -2] #weights for strongBuy, buy, hold, sell,	strongSell advisors respectively
TIMEZONE_WEIGHTS = [4, 3, 2, 1] # weights for 0, -1, -2, -3rd months respectively
ADVICE_COUNT = 10 #how many of best scored symbols we want to get data

In [20]:
symbols = get_symbols()
symbol = yf.Ticker('XTLB')
recommendations = symbol.recommendations
recommendations = recommendations.drop('period', axis='columns')
recommendations

,strongBuy,buy,hold,sell,strongSell
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0


In [21]:
def create_weigth_kernel(vertical_weights, horizontal_weights):
    kernel_values = np.outer(vertical_weights, horizontal_weights)
    kernel = pd.DataFrame(kernel_values)
    kernel.rename(columns={0:'strongBuy', 1:'buy', 2:'hold', 
                              3:'sell', 4:'strongSell'}, inplace = True)
    
    return kernel

In [25]:
def recommendation_score(recommendations, weight_kernel):
    #returns score and how many anaylst we have as advisor in total
    sum = recommendations.to_numpy().sum()
    if sum == 0:
        return None, None
    weighted_sum = (weight_kernel * recommendations).to_numpy().sum()
    return weighted_sum / sum, sum 

In [26]:
def evaluate_symbols(symbols, timezone_weights, recommendation_weights):
    #
    scores = {}
    advisors = {}
    weight_kernel = create_weigth_kernel(timezone_weights, recommendation_weights)
    
    for symbol in symbols:
        recommendations = yf.Ticker(symbol).recommendations
        recommendations = recommendations.drop('period', axis='columns')
        
        score, advisor = recommendation_score(recommendations, weight_kernel)
        if score:  
            scores[symbol] = score
            advisors[symbol] = advisor
        else: 
            print(f"No advisors found for {symbol}.") 
            
    sorted_scores = dict(sorted(scores.items(), key=lambda x: x[1]))
    
    return sorted_scores, advisors  

In [28]:
scores, advisors = evaluate_symbols(symbols, TIMEZONE_WEIGHTS, RECOMMENDATION_WEIGHTS)
print(advisors)
#TODO reverse sort

No advisors found for YHGJ.
No advisors found for XTLB.
{'AAPL': 164, 'MSFT': 197, 'XWEL': 2, 'YJ': 2, 'ZEUS': 8, 'IBM': 85, 'NFLX': 174, 'V': 158, 'CRM': 186, 'DIS': 127, 'TSLA': 154, 'BA': 113, 'AMZN': 215, 'INTC': 173, 'BABA': 194, 'HD': 144, 'PYPL': 179, 'NKE': 153, 'XOM': 103, 'CSCO': 107, 'MRK': 104, 'UNH': 103}
